In [1]:
import pandas as pd
import numpy as np
# view all columns
pd.set_option('display.max_columns', None)
#false positive warning, will not display
pd.options.mode.chained_assignment = None  # default='warn'

In [281]:
# open combined daily and monthly file
df = pd.read_csv("../oklahoma_earthquakes_largefiles/daily_monthly_combined.csv", low_memory=False)

In [282]:
df = df.sort_values(["year"])

In [290]:
df

,API,API_year,APIyear,year,Operator_Name,Operator_Number,Well_Name,Well_Number,Latitude,Longitude,County,Total_Depth,Formation_Name,Jan_Vol,Feb_Vol,Mar_Vol,Apr_Vol,May_Vol,Jun_Vol,Jul_Vol,Aug_Vol,Sep_Vol,Oct_Vol,Nov_Vol,Dec_Vol,year_volume,data_origin,APIyear
16145,3501520498,3501520498-2011,35015204982011,2011,BINGER OPERATIONS LLC,NaN,EAST BINGER UNIT,60GI-1,35.296000,-98.308667,CADDO,10195.0,MARCHAND,13926.0,14357.0,14319.0,15528.0,14990.0,14955.0,14093.0,14987.0,14189.0,14127.00000,15621.00000,13591.00000,174683.00000,monthly,35015204982011
19331,3504900205,3504900205-2011,35049002052011,2011,DAYLIGHT PETROLEUM LLC,NaN,NE PURDY SPRINGER SAND UNIT A (DUNC.ALC.P15W),P15,34.759862,-97.564590,GARVIN,9180.0,SPRINGER SAND,9072.0,14960.0,0.0,111723.0,4522.0,58241.0,13208.0,9547.0,0.0,0.00000,0.00000,0.00000,221273.00000,monthly,35049002052011
19330,3504724528,3504724528-2011,35047245282011,2011,SANDRIDGE EXPLORATION & PRODUCTION LLC,NaN,LOMA SWD,1-1,36.504361,-97.890361,GARFIELD,8500.0,ARBUCKLE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3189.00000,71226.00000,38259.00000,112674.00000,monthly,35047245282011
19329,3504724516,3504724516-2011,35047245162011,2011,SK PLYMOUTH LLC,NaN,TOEWS,1-15 SWD,36.562985,-97.617946,GARFIELD,7503.0,ARBUCKLE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39676.89063,35082.08984,56822.87109,131581.85156,monthly,35047245162011
19328,3504724485,3504724485-2011,35047244852011,2011,KIRKPATRICK OIL COMPANY INC,NaN,MARSHALL EAST SWD,1,36.165073,-97.602745,GARFIELD,7439.0,ARBUCKLE|WILCOX,19769.0,20939.0,42513.0,28752.0,30183.0,29888.0,30605.0,27052.0,25727.0,27502.00000,25197.00000,25491.00000,333618.00000,monthly,35047244852011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4675,3512520225,3512520225-2021,35125202252021,2021,CIRCLE 9 RESOURCES LLC,91814.0,PORTER,2,35.270002,-96.781653,NaN,NaN,Arbuckle,142711.0,73693.0,142837.0,127188.0,141549.0,143675.0,47536.0,0.0,0.0,0.00000,0.00000,0.00000,819189.00000,daily,35125202252021
4674,3512520208,3512520208-2021,35125202082021,2021,PHOENIX OIL & GAS INC,9425.0,DEATHARAGE,B-1,35.056390,-96.889080,NaN,NaN,Arbuckle,156498.0,98046.0,108616.0,162210.0,172546.0,163800.0,55660.0,0.0,0.0,0.00000,0.00000,0.00000,917376.00000,daily,35125202082021
4673,3512500229,3512500229-2021,35125002292021,2021,LAIRD BILL OIL COMPANY,4165.0,MCCURRY,1,35.021185,-96.926533,NaN,NaN,Arbuckle,1860.0,1680.0,1860.0,1800.0,1560.0,1500.0,500.0,0.0,0.0,0.00000,0.00000,0.00000,10760.00000,daily,35125002292021
4679,3512520827,3512520827-2021,35125208272021,2021,ARMOR SWD INC,95305.0,SALES,22-1,34.979280,-96.863414,NaN,NaN,Arbuckle,0.0,0.0,0.0,240.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,240.00000,daily,35125208272021


In [284]:
len(df.Formation_Name.unique())

1269

In [285]:
# Add API_year to columns
df["APIyear"] = df.apply(
        lambda row: row.API_year[0:10] + row.API_year[-4:], axis = 1)
df.APIyear = df.APIyear.astype(int)

In [286]:
df.columns

Index(['API', 'API_year', 'year', 'Operator_Name', 'Operator_Number',
       'Well_Name', 'Well_Number', 'Latitude', 'Longitude', 'County',
       'Total_Depth', 'Formation_Name', 'Jan_Vol', 'Feb_Vol', 'Mar_Vol',
       'Apr_Vol', 'May_Vol', 'Jun_Vol', 'Jul_Vol', 'Aug_Vol', 'Sep_Vol',
       'Oct_Vol', 'Nov_Vol', 'Dec_Vol', 'year_volume', 'data_origin',
       'APIyear'],
      dtype='object')

In [287]:
# Reorder columns
df = df[['API', 'API_year','APIyear', 'year', 'Operator_Name', 'Operator_Number',
       'Well_Name', 'Well_Number', 'Latitude', 'Longitude', 'County',
       'Total_Depth', 'Formation_Name', 'Jan_Vol', 'Feb_Vol', 'Mar_Vol',
       'Apr_Vol', 'May_Vol', 'Jun_Vol', 'Jul_Vol', 'Aug_Vol', 'Sep_Vol',
       'Oct_Vol', 'Nov_Vol', 'Dec_Vol', 'year_volume', 'data_origin',
       'APIyear']]

In [309]:
# Get list of AOI wells
aoi_wells_df = df[df.data_origin == "daily"]
aoi_wells = aoi_wells_df.API.unique()
# len(aoi_wells)

In [310]:
# Filter out non-aoi wells from dataframe (i.e., filters them out from the monthly wells)
df_filtered = df[df['API'].isin(aoi_wells)]
len(df_filtered)

11848

In [311]:
aoi_wells_df.Formation_Name.unique()

array(['Arbuckle'], dtype=object)

In [312]:
df_filtered.Formation_Name.unique()

array(['ARBUCKLE', 'ARBUCKLE|WILCOX', nan, 'ARBUCKLE GROUP',
       'ARBUCKLE DOLOMITE', 'WILCOX', 'TYNER', 'BURBANK',
       'ARBUCKLE|SEE ORIGINAL', 'ARBUCKLE|PENNSYLVANIAN|SIMPSON',
       'ARBUCKLE LIME',
       'ARBUCKLE|BIG LIME|CHEROKEE|CLEVELAND|ENDICOTT|LAYTON|MISSISSIPPI CHERT|MISSISSIPPIAN|OSWEGO|PAWHUSKA|SIMPSON|SKINNER|TONKAWA|WOODFORD',
       'ARBUCKLE|CHATTANOOGA|CHECKERBOARD|CHEROKEE|ENDICOTT|HOOVER|LAYTON|MISSISSIPPIAN|OSWEGO|PAWHUSKA|SIMPSON|TONKAWA',
       'ARBUCKLE|MISSISSIPPI|WILCOX', 'SIMPSON',
       '1ST WILCOX|2ND WILCOX|ARBUCKLE|ARBUCKLE DOL|HUNTON LS|MCLISH FM|OIL CREEK ZONE|SYLVAN SH|VIOLA LS|WOODFORD SH',
       'ARBUCKLE|HUNTON', 'PRUE', 'MISSISSIPPI',
       '1ST WILCOX|2ND WILCOX|ARBUCKLE|HUNTON LM.|OSWEGO LM.|VIOLA LM.',
       'HUNTON', 'BIG LIME-OSWEGO|MISSISSIPPIAN SOLID',
       'ARBUCKLE|CHECKERBOARD|CHEROKEE|ENDICOTT|LAYTON|MISSISSIPPIAN|OSWEGO|PAWHUSKA|SIMPSON|TONKAWA|WOODFORD',
       'MISENER',
       'ARBUCKLE|BOOCH|CROMWELL|GILCREASE|HUNTON

In [313]:
df_dupes = df_filtered[df_filtered.APIyear.duplicated(keep=False)]

In [314]:
df_dupes = df_dupes.sort_values(by = ["year", "API_year"])
print(len(df_dupes))

7988


In [315]:
df_dupes

,API,API_year,APIyear,year,Operator_Name,Operator_Number,Well_Name,Well_Number,Latitude,Longitude,County,Total_Depth,Formation_Name,Jan_Vol,Feb_Vol,Mar_Vol,Apr_Vol,May_Vol,Jun_Vol,Jul_Vol,Aug_Vol,Sep_Vol,Oct_Vol,Nov_Vol,Dec_Vol,year_volume,data_origin,APIyear
59220,3500322020,3500322020-2014,35003220202014,2014,SANDRIDGE EXPLORATION & PRODUCTION LLC,NaN,ROSE KELLY SWD,1-3,36.928442,-98.256115,ALFALFA,6842.0,ARBUCKLE,119860.0,109017.0,111605.0,98214.0,107480.0,106385.0,88927.0,139751.0,109548.0,104237.0,103687.0,104595.0,1303306.0,monthly,35003220202014
0,3500322020,3500322020-2014,35003220202014,2014,SANDRIDGE EXPLORATION & PRODUCTION LLC,20357.0,ROSE KELLY SWD,1-3,36.928442,-98.256115,NaN,NaN,Arbuckle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9948.0,104595.0,114543.0,daily,35003220202014
59225,3500322028,3500322028-2014,35003220282014,2014,SANDRIDGE EXPLORATION & PRODUCTION LLC,NaN,ROXY SWD,1-30,36.957722,-98.314854,ALFALFA,8310.0,ARBUCKLE,615298.0,527924.0,600735.0,557427.0,575356.0,542243.0,537129.0,408907.0,476038.0,437853.0,500791.0,530498.0,6310199.0,monthly,35003220282014
1,3500322028,3500322028-2014,35003220282014,2014,SANDRIDGE EXPLORATION & PRODUCTION LLC,20357.0,ROXY SWD,1-30,36.957722,-98.314854,NaN,NaN,Arbuckle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46364.0,530498.0,576862.0,daily,35003220282014
59232,3500322131,3500322131-2014,35003221312014,2014,SANDRIDGE EXPLORATION & PRODUCTION LLC,NaN,ASHLEY SWD,1-36,36.868800,-98.231700,ALFALFA,7450.0,ARBUCKLE,300691.0,191887.0,191989.0,165472.0,175464.0,249733.0,321752.0,269494.0,283704.0,370705.0,150240.0,299450.0,2970581.0,monthly,35003221312014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59151,3515124298,3515124298-2020,35151242982020,2020,SANDRIDGE EXPLORATION & PRODUCTION LLC,NaN,SOLO SWD 2815,1-16,36.900048,-98.823159,WOODS,6730.0,ARBUCKLE,45158.0,47507.0,40477.0,41243.0,35395.0,28982.0,28539.0,28854.0,27478.0,31192.0,31751.0,33099.0,419675.0,monthly,35151242982020
1534,3515124369,3515124369-2020,35151243692020,2020,AMPLIFY OKLAHOMA OPERATING LLC,85115.0,MEIER 2614,1-21 SWD,36.711125,-98.713202,NaN,NaN,Arbuckle,405505.0,297183.0,375937.0,367886.0,347353.0,234826.0,234005.0,222450.0,180167.0,84031.0,34738.0,105399.0,2889480.0,daily,35151243692020
59152,3515124369,3515124369-2020,35151243692020,2020,AMPLIFY OKLAHOMA OPERATING LLC,NaN,MEIER 2614,1-21 SWD,36.711125,-98.713202,WOODS,10636.0,ARBUCKLE,405057.0,296441.0,372108.0,367018.0,343238.0,235233.0,230637.0,222577.0,178011.0,81607.0,34350.0,106103.0,2872380.0,monthly,35151243692020
1535,3515323010,3515323010-2020,35153230102020,2020,TAPSTONE ENERGY LLC,146819.0,MULLINS SWD,1-33,36.422972,-99.015241,NaN,NaN,Arbuckle,52863.0,52744.0,55165.0,41433.0,27977.0,33166.0,43381.0,41794.0,48516.0,50547.0,36923.0,21358.0,505867.0,daily,35153230102020


In [316]:
df_dupes_daily = df_dupes[df_dupes.data_origin == "daily"]

In [317]:
df_dupes_daily.year_volume.sum()

3193029590.021061

In [318]:
df_dupes_monthly = df_dupes[df_dupes.data_origin == "monthly"]

In [319]:
df_dupes_monthly.year_volume.sum()

3437634226.767399

In [320]:
(df_dupes_monthly.year_volume.sum()-df_dupes_daily.year_volume.sum())/df_dupes_daily.year_volume.sum()*100

7.660581584047408

In [321]:
df_no_dupes = df_filtered.drop_duplicates(subset = ["APIyear"], keep = False)

In [323]:
df_no_dupes

,API,API_year,APIyear,year,Operator_Name,Operator_Number,Well_Name,Well_Number,Latitude,Longitude,County,Total_Depth,Formation_Name,Jan_Vol,Feb_Vol,Mar_Vol,Apr_Vol,May_Vol,Jun_Vol,Jul_Vol,Aug_Vol,Sep_Vol,Oct_Vol,Nov_Vol,Dec_Vol,year_volume,data_origin,APIyear
19330,3504724528,3504724528-2011,35047245282011,2011,SANDRIDGE EXPLORATION & PRODUCTION LLC,NaN,LOMA SWD,1-1,36.504361,-97.890361,GARFIELD,8500.0,ARBUCKLE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3189.00000,71226.00000,38259.00000,112674.00000,monthly,35047245282011
19329,3504724516,3504724516-2011,35047245162011,2011,SK PLYMOUTH LLC,NaN,TOEWS,1-15 SWD,36.562985,-97.617946,GARFIELD,7503.0,ARBUCKLE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39676.89063,35082.08984,56822.87109,131581.85156,monthly,35047245162011
19328,3504724485,3504724485-2011,35047244852011,2011,KIRKPATRICK OIL COMPANY INC,NaN,MARSHALL EAST SWD,1,36.165073,-97.602745,GARFIELD,7439.0,ARBUCKLE|WILCOX,19769.0,20939.0,42513.0,28752.0,30183.0,29888.0,30605.0,27052.0,25727.0,27502.00000,25197.00000,25491.00000,333618.00000,monthly,35047244852011
19327,3504724444,3504724444-2011,35047244442011,2011,ARROWHEAD ENERGY INC,NaN,VENCL SWD,2-26,36.532275,-97.696179,GARFIELD,7212.0,NaN,7440.0,6720.0,7440.0,7200.0,7440.0,7200.0,7440.0,7440.0,7200.0,7440.00000,7200.00000,7440.00000,87600.00000,monthly,35047244442011
19326,3504724404,3504724404-2011,35047244042011,2011,M M ENERGY INC,NaN,GREGG,1,36.355337,-97.562891,GARFIELD,6000.0,NaN,24440.0,21785.0,24840.0,23400.0,22175.0,12950.0,25790.0,24710.0,26370.0,28190.00000,27800.00000,29140.00000,291590.00000,monthly,35047244042011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4675,3512520225,3512520225-2021,35125202252021,2021,CIRCLE 9 RESOURCES LLC,91814.0,PORTER,2,35.270002,-96.781653,NaN,NaN,Arbuckle,142711.0,73693.0,142837.0,127188.0,141549.0,143675.0,47536.0,0.0,0.0,0.00000,0.00000,0.00000,819189.00000,daily,35125202252021
4674,3512520208,3512520208-2021,35125202082021,2021,PHOENIX OIL & GAS INC,9425.0,DEATHARAGE,B-1,35.056390,-96.889080,NaN,NaN,Arbuckle,156498.0,98046.0,108616.0,162210.0,172546.0,163800.0,55660.0,0.0,0.0,0.00000,0.00000,0.00000,917376.00000,daily,35125202082021
4673,3512500229,3512500229-2021,35125002292021,2021,LAIRD BILL OIL COMPANY,4165.0,MCCURRY,1,35.021185,-96.926533,NaN,NaN,Arbuckle,1860.0,1680.0,1860.0,1800.0,1560.0,1500.0,500.0,0.0,0.0,0.00000,0.00000,0.00000,10760.00000,daily,35125002292021
4679,3512520827,3512520827-2021,35125208272021,2021,ARMOR SWD INC,95305.0,SALES,22-1,34.979280,-96.863414,NaN,NaN,Arbuckle,0.0,0.0,0.0,240.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,240.00000,daily,35125208272021


In [324]:
df_no_dupes[df_no_dupes.data_origin == "monthly"].count()

API                3020
API_year           3020
APIyear            3020
year               3020
Operator_Name      3020
Operator_Number       0
Well_Name          3020
Well_Number        3020
Latitude           3020
Longitude          3020
County             3020
Total_Depth        2935
Formation_Name     2272
Jan_Vol            3020
Feb_Vol            3020
Mar_Vol            3020
Apr_Vol            3020
May_Vol            3020
Jun_Vol            3020
Jul_Vol            3020
Aug_Vol            3020
Sep_Vol            3020
Oct_Vol            3020
Nov_Vol            3020
Dec_Vol            3020
year_volume        3020
data_origin        3020
APIyear            3020
dtype: int64

In [325]:
df_no_dupes[df_no_dupes.data_origin == "daily"].count()

API                840
API_year           840
APIyear            840
year               840
Operator_Name      840
Operator_Number    840
Well_Name          840
Well_Number        840
Latitude           840
Longitude          840
County               0
Total_Depth          0
Formation_Name     840
Jan_Vol            840
Feb_Vol            840
Mar_Vol            840
Apr_Vol            840
May_Vol            840
Jun_Vol            840
Jul_Vol            840
Aug_Vol            840
Sep_Vol            840
Oct_Vol            840
Nov_Vol            840
Dec_Vol            840
year_volume        840
data_origin        840
APIyear            840
dtype: int64

In [182]:
wells_in_aoi = df_no_dupes[df_no_dupes.data_origin == "daily"]

In [189]:
wells_in_aoi_list = df_no_dupes.API.unique()

In [192]:
len(wells_in_aoi_list)

943

In [326]:
df_round_low = df_no_dupes.append(df_dupes_daily, ignore_index = True)

In [327]:
len(df_round_low.API.unique())

833

In [177]:
df_round_low.head()

,API,API_year,APIyear,year,Operator_Name,Operator_Number,Well_Name,Well_Number,Latitude,Longitude,County,Total_Depth,Formation_Name,Jan_Vol,Feb_Vol,Mar_Vol,Apr_Vol,May_Vol,Jun_Vol,Jul_Vol,Aug_Vol,Sep_Vol,Oct_Vol,Nov_Vol,Dec_Vol,year_volume,data_origin,APIyear
0,3500322308,3500322308-2014,35003223082014,2014,EAGLE EXPLORATION PRODUCTION LLC,103155.0,REITZ,1 SWD,36.840300,-98.361100,NaN,NaN,arbuckle,359436.0,320423.0,452791.0,473125.0,661878.0,552944.0,594604.0,413017.0,223618.0,375457.0,423063.0,374999.0,5225355.0,daily,35003223082014
1,3500322521,3500322521-2014,35003225212014,2014,SANDRIDGE EXPLORATION & PRODUCTION LLC,20357.0,CLARK W SWD 2811,1-27,36.871070,-98.371700,NaN,NaN,arbuckle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47650.0,407170.0,454820.0,daily,35003225212014
2,3500322623,3500322623-2014,35003226232014,2014,SANDRIDGE EXPLORATION & PRODUCTION LLC,20357.0,PHIN SWD 2811,1-11,36.914120,-98.352580,NaN,NaN,arbuckle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39370.0,524007.0,563377.0,daily,35003226232014
3,3500322903,3500322903-2014,35003229032014,2014,EAGLE EXPLORATION PRODUCTION LLC,103155.0,REITZ,2 SWD,36.840195,-98.361295,NaN,NaN,arbuckle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275634.0,436316.0,286431.0,218255.0,171164.0,1387800.0,daily,35003229032014
4,3504724485,3504724485-2014,35047244852014,2014,KIRKPATRICK OIL COMPANY INC,17881.0,MARSHALL EAST SWD,1,36.165073,-97.602481,NaN,NaN,arbuckle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11808.0,12216.0,8784.0,32439.0,38763.0,104010.0,daily,35047244852014


In [328]:
df_round_low.to_csv("/Users/curtissmith/Projects/oklahoma_earthquakes_largefiles/python_exports/df_round_low.csv", index = False)

In [329]:
df_round_high = df_no_dupes.append(df_dupes_monthly, ignore_index = True)

In [330]:
df_round_high.to_csv("/Users/curtissmith/Projects/oklahoma_earthquakes_largefiles/python_exports/df_round_high.csv", index = False)

In [181]:
df_round_low.Formation_Name.unique()

array(['arbuckle'], dtype=object)

In [278]:
df_round_high.Formation_Name.unique()

array(['arbuckle'], dtype=object)

In [280]:
df_round_low

,API,API_year,APIyear,year,Operator_Name,Operator_Number,Well_Name,Well_Number,Latitude,Longitude,County,Total_Depth,Formation_Name,Jan_Vol,Feb_Vol,Mar_Vol,Apr_Vol,May_Vol,Jun_Vol,Jul_Vol,Aug_Vol,Sep_Vol,Oct_Vol,Nov_Vol,Dec_Vol,year_volume,data_origin,APIyear
0,3500322308,3500322308-2014,35003223082014,2014,EAGLE EXPLORATION PRODUCTION LLC,103155.0,REITZ,1 SWD,36.840300,-98.361100,NaN,NaN,arbuckle,359436.00000,320423.00000,452791.0,473125.0,661878.0,552944.0,594604.0,413017.0,223618.0,375457.0,423063.0,374999.0,5.225355e+06,daily,35003223082014
1,3500322521,3500322521-2014,35003225212014,2014,SANDRIDGE EXPLORATION & PRODUCTION LLC,20357.0,CLARK W SWD 2811,1-27,36.871070,-98.371700,NaN,NaN,arbuckle,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47650.0,407170.0,4.548200e+05,daily,35003225212014
2,3500322623,3500322623-2014,35003226232014,2014,SANDRIDGE EXPLORATION & PRODUCTION LLC,20357.0,PHIN SWD 2811,1-11,36.914120,-98.352580,NaN,NaN,arbuckle,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39370.0,524007.0,5.633770e+05,daily,35003226232014
3,3500322903,3500322903-2014,35003229032014,2014,EAGLE EXPLORATION PRODUCTION LLC,103155.0,REITZ,2 SWD,36.840195,-98.361295,NaN,NaN,arbuckle,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,275634.0,436316.0,286431.0,218255.0,171164.0,1.387800e+06,daily,35003229032014
4,3504724485,3504724485-2014,35047244852014,2014,KIRKPATRICK OIL COMPANY INC,17881.0,MARSHALL EAST SWD,1,36.165073,-97.602481,NaN,NaN,arbuckle,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,11808.0,12216.0,8784.0,32439.0,38763.0,1.040100e+05,daily,35047244852014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6847,3515124098,3515124098-2016,35151240982016,2016,CHAPARRAL ENERGY LLC,15003.0,FREEDOM,26 SWD,36.699122,-98.898064,NaN,NaN,arbuckle,9443.00000,8374.00000,7177.0,10088.0,11272.0,9138.0,12828.0,16598.0,18535.0,17862.0,13928.0,12579.0,1.478220e+05,daily,35151240982016
6848,3515124258,3515124258-2016,35151242582016,2016,SANDRIDGE EXPLORATION & PRODUCTION LLC,20357.0,SEEBE SWD 2713,2-23,36.798458,-98.561324,NaN,NaN,arbuckle,409903.00000,364249.00000,446048.0,403498.0,388251.0,358221.0,351911.0,361086.0,344550.0,345426.0,339078.0,349482.0,4.461703e+06,daily,35151242582016
6849,3515124293,3515124293-2016,35151242932016,2016,AMPLIFY OKLAHOMA OPERATING LLC,85115.0,DENTON SWD,1-28,36.622611,-98.597493,NaN,NaN,arbuckle,784250.00000,523340.00000,572840.0,537806.0,700126.0,632246.0,630546.0,546795.0,562940.0,532409.0,533155.0,525088.0,7.081541e+06,daily,35151242932016
6850,3515124298,3515124298-2016,35151242982016,2016,SANDRIDGE EXPLORATION & PRODUCTION LLC,20357.0,SOLO SWD 2815,1-16,36.900048,-98.823159,NaN,NaN,arbuckle,0.00000,116732.00000,284741.0,230205.0,231847.0,210014.0,200839.0,205193.0,210626.0,255777.0,235894.0,212277.0,2.394145e+06,daily,35151242982016


In [279]:
df_round_high

,API,API_year,APIyear,year,Operator_Name,Operator_Number,Well_Name,Well_Number,Latitude,Longitude,County,Total_Depth,Formation_Name,Jan_Vol,Feb_Vol,Mar_Vol,Apr_Vol,May_Vol,Jun_Vol,Jul_Vol,Aug_Vol,Sep_Vol,Oct_Vol,Nov_Vol,Dec_Vol,year_volume,data_origin,APIyear
0,3500322308,3500322308-2014,35003223082014,2014,EAGLE EXPLORATION PRODUCTION LLC,103155.0,REITZ,1 SWD,36.840300,-98.361100,NaN,NaN,arbuckle,359436.0,320423.0,452791.0,473125.0,661878.0,552944.0,594604.0,413017.0,223618.0,375457.0,423063.0,374999.0,5225355.0,daily,35003223082014
1,3500322521,3500322521-2014,35003225212014,2014,SANDRIDGE EXPLORATION & PRODUCTION LLC,20357.0,CLARK W SWD 2811,1-27,36.871070,-98.371700,NaN,NaN,arbuckle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47650.0,407170.0,454820.0,daily,35003225212014
2,3500322623,3500322623-2014,35003226232014,2014,SANDRIDGE EXPLORATION & PRODUCTION LLC,20357.0,PHIN SWD 2811,1-11,36.914120,-98.352580,NaN,NaN,arbuckle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39370.0,524007.0,563377.0,daily,35003226232014
3,3500322903,3500322903-2014,35003229032014,2014,EAGLE EXPLORATION PRODUCTION LLC,103155.0,REITZ,2 SWD,36.840195,-98.361295,NaN,NaN,arbuckle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,275634.0,436316.0,286431.0,218255.0,171164.0,1387800.0,daily,35003229032014
4,3504724485,3504724485-2014,35047244852014,2014,KIRKPATRICK OIL COMPANY INC,17881.0,MARSHALL EAST SWD,1,36.165073,-97.602481,NaN,NaN,arbuckle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11808.0,12216.0,8784.0,32439.0,38763.0,104010.0,daily,35047244852014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6847,3515124098,3515124098-2018,35151240982018,2018,CHAPARRAL ENERGY LLC,NaN,FREEDOM,26 SWD,36.699122,-98.898064,WOODS,8400.0,arbuckle,8138.0,8980.0,10394.0,8905.0,9051.0,8574.0,8951.0,7703.0,7708.0,8069.0,7240.0,7041.0,100754.0,monthly,35151240982018
6848,3515124258,3515124258-2018,35151242582018,2018,SANDRIDGE EXPLORATION & PRODUCTION LLC,NaN,SEEBE SWD 2713,2-23,36.798458,-98.561324,WOODS,7464.0,arbuckle,283934.0,269518.0,275416.0,264231.0,278467.0,255238.0,276066.0,291595.0,263376.0,310205.0,311946.0,327131.0,3407123.0,monthly,35151242582018
6849,3515124293,3515124293-2018,35151242932018,2018,AMPLIFY OKLAHOMA OPERATING LLC,NaN,DENTON SWD,1-28,36.622611,-98.597493,WOODS,7440.0,arbuckle,158627.0,249868.0,429233.0,243962.0,426023.0,350720.0,427970.0,384266.0,277348.0,260995.0,253680.0,78200.0,3540892.0,monthly,35151242932018
6850,3515124298,3515124298-2018,35151242982018,2018,SANDRIDGE EXPLORATION & PRODUCTION LLC,NaN,SOLO SWD 2815,1-16,36.900048,-98.823159,WOODS,6730.0,arbuckle,122642.0,105426.0,110789.0,97156.0,97212.0,93934.0,86105.0,76850.0,78334.0,72842.0,67152.0,68207.0,1076649.0,monthly,35151242982018


In [332]:
df_filtered_monthly = df_filtered[df_filtered.data_origin == "monthly"]

In [333]:
df_filtered_monthly.to_csv("/Users/curtissmith/Projects/oklahoma_earthquakes_largefiles/python_exports/df_filtered_monthly.csv", index = False)

In [334]:
filtered_api_list = df_filtered_monthly.API.unique()

In [336]:
# test_list = []
# for i in filtered_api_list:
#     if i not in aoi_wells:
#         test_list.append(i)
        
test_list = [i for i in filtered_api_list if i not in aoi_wells]        

In [337]:
test_list

[]

In [338]:
len(filtered_api_list)

832

In [339]:
len(aoi_wells)

833

In [341]:
df_filtered_monthly

,API,API_year,APIyear,year,Operator_Name,Operator_Number,Well_Name,Well_Number,Latitude,Longitude,County,Total_Depth,Formation_Name,Jan_Vol,Feb_Vol,Mar_Vol,Apr_Vol,May_Vol,Jun_Vol,Jul_Vol,Aug_Vol,Sep_Vol,Oct_Vol,Nov_Vol,Dec_Vol,year_volume,data_origin,APIyear
19330,3504724528,3504724528-2011,35047245282011,2011,SANDRIDGE EXPLORATION & PRODUCTION LLC,NaN,LOMA SWD,1-1,36.504361,-97.890361,GARFIELD,8500.0,ARBUCKLE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3189.00000,71226.00000,38259.00000,112674.00000,monthly,35047245282011
19329,3504724516,3504724516-2011,35047245162011,2011,SK PLYMOUTH LLC,NaN,TOEWS,1-15 SWD,36.562985,-97.617946,GARFIELD,7503.0,ARBUCKLE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39676.89063,35082.08984,56822.87109,131581.85156,monthly,35047245162011
19328,3504724485,3504724485-2011,35047244852011,2011,KIRKPATRICK OIL COMPANY INC,NaN,MARSHALL EAST SWD,1,36.165073,-97.602745,GARFIELD,7439.0,ARBUCKLE|WILCOX,19769.0,20939.0,42513.0,28752.0,30183.0,29888.0,30605.0,27052.0,25727.0,27502.00000,25197.00000,25491.00000,333618.00000,monthly,35047244852011
19327,3504724444,3504724444-2011,35047244442011,2011,ARROWHEAD ENERGY INC,NaN,VENCL SWD,2-26,36.532275,-97.696179,GARFIELD,7212.0,NaN,7440.0,6720.0,7440.0,7200.0,7440.0,7200.0,7440.0,7440.0,7200.0,7440.00000,7200.00000,7440.00000,87600.00000,monthly,35047244442011
19326,3504724404,3504724404-2011,35047244042011,2011,M M ENERGY INC,NaN,GREGG,1,36.355337,-97.562891,GARFIELD,6000.0,NaN,24440.0,21785.0,24840.0,23400.0,22175.0,12950.0,25790.0,24710.0,26370.0,28190.00000,27800.00000,29140.00000,291590.00000,monthly,35047244042011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51867,3503729201,3503729201-2020,35037292012020,2020,BAUGH ALLAN L,NaN,FOUNTAIN SWD,1,35.902091,-96.584933,CREEK,4300.0,ARBUCKLE,0.0,0.0,0.0,0.0,0.0,0.0,4020.0,2990.0,1900.0,2270.00000,3140.00000,3445.00000,17765.00000,monthly,35037292012020
51869,3503729213,3503729213-2020,35037292132020,2020,CIMARRON RIVER OPERATING CO,NaN,E. YAHOLA,2A,36.057570,-96.596340,CREEK,3450.0,ARBUCKLE,4450.0,40369.0,41212.0,40127.0,41168.0,36634.0,39317.0,39384.0,36742.0,38065.00000,39133.00000,39818.00000,436419.00000,monthly,35037292132020
51871,3503729215,3503729215-2020,35037292152020,2020,MID-CON ENERGY OPERATING LLC,NaN,MCBRIDE,35,36.050040,-96.599400,CREEK,3400.0,ARBUCKLE,215120.0,181020.0,95300.0,45220.0,90330.0,47310.0,40980.0,50920.0,37460.0,43550.00000,42340.00000,40060.00000,929610.00000,monthly,35037292152020
51905,3503736202,3503736202-2020,35037362022020,2020,PETCO PETROLEUM CORP,NaN,BEN WACOCHE,4,36.027504,-96.598293,CREEK,2680.0,ARBUCKLE,91791.0,78809.0,90007.0,90098.0,78189.0,75234.0,75509.0,0.0,0.0,0.00000,0.00000,0.00000,579637.00000,monthly,35037362022020
